In [1]:
import warnings

warnings.filterwarnings(action='ignore')

# BCE Loss를 이용한 Logistic-Regression (이진 분류) 

- BCE를 손실함수로 사용하고 경사하강법을 이용하여 모델의 가중치를 업데이트

- 경사하강법은 SGD가 아닌 전체 데이터를 사용하는 Batch GD를 사용함

- Y_hat = sigmoid(WX + b) * threshold = 0.5

- Loss = BCE = -(Y * log(Y_hat) + ((1 - Y) * log(1 - Y_hat))) / N

In [2]:
import numpy as np


def sigmoid(X):
    return 1 / (1 + np.exp(-X))

threshold = 0.5

In [3]:
# 데이터 생성

X = np.random.randn(1000, 10)
W = np.random.rand(10)
b = np.random.rand()

Y = X.dot(W) + b
Y = sigmoid(Y)
Y = (Y >= threshold).astype(int)

In [4]:
class LogisticRegressionBCE:
    def __init__(self, input_dim, lr, epochs, seed):
        self.input_dim = input_dim
        self.lr = lr
        self.epochs = epochs
        np.random.seed(seed)
        self.W = np.random.rand(input_dim)
        self.b = 0.0
    
    def backword(self, X, Y, Y_hat):
        dE = (Y_hat - Y).reshape(-1, 1) # (data, 1)
        dW = np.mean(dE * X, axis = 0) # (data, input_dim) -> (1, input_dim)
        db = np.mean(dE) # (data, 1) -> (1,)

        self.W -= dW * self.lr
        self.b -= db * self.lr

    def forword(self, X, threshold = 0.5):
        Y_hat = sigmoid(X.dot(self.W) + self.b)
        Y_hat = (Y_hat >= threshold).astype(int)
        return Y_hat

    def loss(self, Y, Y_hat):
        epsilon = 1e-10
        return -np.mean(Y * np.log(Y_hat + epsilon) + (1 - Y) * np.log((1 - Y_hat) + epsilon))

    def fit(self, X, Y):
        '''
        X = (data, input_dim)
        Y = (data, 1)
        '''

        for epoch in range(1, self.epochs + 1):
            Y_hat = self.forword(X)
            error = self.loss(Y, Y_hat)
            if epoch % 1000 == 0:
                print(f'Epoch: {epoch} | Loss : {error} | Acc : {sum(Y_hat == Y) / len(Y)}')
            self.backword(X, Y, Y_hat)

In [5]:
model = LogisticRegressionBCE(input_dim = X.shape[1], lr = 0.01, epochs = 5000, seed = 22)

In [6]:
model.fit(X, Y)

Epoch: 1000 | Loss : 0.34538776385060704 | Acc : 0.985
Epoch: 2000 | Loss : 0.04605170176008093 | Acc : 0.998
Epoch: 3000 | Loss : 0.06907755269012142 | Acc : 0.997
Epoch: 4000 | Loss : -1.0000000826903712e-10 | Acc : 1.0
Epoch: 5000 | Loss : -1.0000000826903712e-10 | Acc : 1.0


In [7]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(penalty="none")
model.fit(X, Y)
print(f"Score:{model.score(X, Y)}")

Score:1.0


학습 결과 target과 가까워지게 W와 b가 업데이트 되지는 않지만(sigmoid를 취한 아웃풋에 임계치를 기반으로 예측을 하기 때문에, 정확한 회귀선을 추정하지 않더라도 값을 표현하는 모델을 만들 수 있음), ACC가 올라간다는 것을 확인할 수 있음

# CE Loss를 이용한 Logistic-Regression (2개 이상의 class 분류) 

- CE를 손실함수로 사용하고 경사하강법을 이용하여 모델의 가중치를 업데이트

- 경사하강법은 SGD가 아닌 전체 데이터를 사용하는 Batch GD를 사용함

- Y_hat = argmax(softmax(WX + b))

- Loss = CE = -(Y * log(Y_hat)) / N

In [8]:
import numpy as np

def softmax(X):
    e_x = np.exp(X)
    return e_x / np.sum(e_x, 1).reshape(-1, 1)

In [9]:
# 데이터 생성
from sklearn.datasets import load_iris

data = load_iris()

X = data.data
Y = data.target
classse = len(data.target_names)

In [31]:
class LogisticRegressionCE:
    def __init__(self, input_dim, classse, lr, epochs, seed):
        self.input_dim = input_dim
        self.classse = classse
        self.lr = lr
        self.epochs = epochs
        np.random.seed(seed)
        self.W = np.random.rand(input_dim, classse)
        self.b = np.zeros(classse)
    
    def backword(self, X, Y, Y_hat):
        dE = (Y_hat - Y) # (data, classse)
        dW = np.dot(X.T, dE) # (input_dim, data).dot(data, classse) -> (input_dim, classse)
        db = np.mean(dE, axis = 0) # (data, classse) -> (classse,)

        self.W -= dW * self.lr
        self.b -= db * self.lr

    def forword(self, X):
        Y_hat = softmax(X.dot(self.W) + self.b)
        return Y_hat

    def loss(self, Y, Y_hat):
        epsilon = 1e-10
        return -np.mean(Y * np.log(Y_hat + epsilon))

    def fit(self, X, Y):
        '''
        X = (data, input_dim)
        Y = (data, 1)
        '''
        Y = np.eye(self.classse)[Y]
        
        for epoch in range(1, self.epochs + 1):
            Y_hat = self.forword(X)
            error = self.loss(Y, Y_hat)
            if epoch % 100 == 0:
                print(f'Epoch: {epoch} | Loss : {error} | Acc : {sum(np.argmax(Y_hat, axis=1) == np.argmax(Y, axis=1)) / len(Y)}')
            self.backword(X, Y, Y_hat)

In [32]:
model = LogisticRegressionCE(input_dim = X.shape[1], classse = classse, lr = 0.0005, epochs = 1000, seed = 22)

In [33]:
model.fit(X, Y)

Epoch: 100 | Loss : 0.129196993813102 | Acc : 0.9533333333333334
Epoch: 200 | Loss : 0.101698627436187 | Acc : 0.9666666666666667
Epoch: 300 | Loss : 0.08614956566334889 | Acc : 0.9666666666666667
Epoch: 400 | Loss : 0.07588508576471162 | Acc : 0.9733333333333334
Epoch: 500 | Loss : 0.06857670197925987 | Acc : 0.9733333333333334
Epoch: 600 | Loss : 0.06310056341759517 | Acc : 0.9733333333333334
Epoch: 700 | Loss : 0.05883938113601851 | Acc : 0.9733333333333334
Epoch: 800 | Loss : 0.05542539628704162 | Acc : 0.9733333333333334
Epoch: 900 | Loss : 0.05262591979334622 | Acc : 0.9733333333333334
Epoch: 1000 | Loss : 0.05028657214104328 | Acc : 0.9733333333333334


In [34]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(penalty="none")
model.fit(X, Y)
print(f"Score:{model.score(X, Y)}")

Score:0.9866666666666667
